# Transition States and Reactivity

## Introduction to the Icing Model

### The Model

#### Lowest Energy of the Icing model

The interaction energy is defined as $- \frac{1}{2} J \sum_i^N \sum_{j\  \in\  \mathrm{neighbours}\left(i\right)} s_i s_j$, where the first sum loops over the number of spins in the system and the second sum loops over the number of neighbours. When keeping all parameters constant apart from the number of dimensions, the only thing changing is the second sum. 

For $D =1$ the number of neighbours is $2$, for $D=2$ it's $4$ and for $D=3$ it's $6$. In general one can say that it the number of neighbours is $2D$. 

In the lowest energy state all spins will be aligned leading to maximum multiplicity and thus a scenario where $s_i = s_j$ for all $i$ and$j$. As the value of $s$ can only have the values $1$ or $-1$, the product under the sum must always equal to $1$.

This interaction energy equation therefore simplifies to $- \frac{1}{2} J \sum_1^N \sum_1^{2D} 1$. The $\frac{1}{2}$ cancels out with the $2$ from $2D$ and once the sums are evaluated they simplify to $E\ =\ -DNJ$. q.e.d.

The entropy of this system is given by $S = k_B \ln W$, where $W$ is the multiplicity. In this system the multiplicity is one as there is only one configuration for a particular state when all spins are aligned. Thus the entropy equals to zero.


## Phase Transitions

### Energy change caused by a change in state and the entropy involved

Using the equation for the interaction energy for the lowest state from above, the lowest energy configuration has a value of $E=-3000 J$.
In order to obtain an energy for the system when one spin is antiparallel to the others one can examine the general equation. The antiparallel spin will occur as $s_i$ once and as $s_j$ 6 times (for 6 different $s_i$ neighbours once). This will lead to terms adding to the energy and terms cancelling out each other eventually leading to a value of $E = -2988 J$. So the energy change is $dE = 12 J$.

The flipping of the spin can happen on any point of the 1000 sites in the lattice. This demonstrates that there are 1000 configurations where one spin is antiparallel to the others. This leads to an entropy value gain of $dS=k_B \ln 1000=9.53 \times 10^{-23}  J  K^{-1}$

### Magnetisation in a 3-D lattice

As there is always one more positive spin in both the 1D and 2D lattice, both systems have a magnetisation of $+1$.

Absolute zero indicates that all spins in a system are aligned, so they either all have an value of $+1$ or $-1$, leading to a total magnetisation of either $+1000$ or $-1000$ respectively.


## Calculating the energy and magnetisation

### Defining the energy() and magnetisation() function


In [ ]:
def energy(self):        
    energy = 0.0
    #copying the boundaries of the lattice and extending/appending them on the opposite ends in all four directions
    new_lattice = np.append(self.lattice, [self.lattice[0]], axis=0)
    new_lattice2 = np.append([self.lattice[-1]], new_lattice, axis = 0)
    new_lattice3 = np.concatenate((np.transpose([new_lattice2[:,-1]]),new_lattice2), axis = 1)
    new_lattice4 = np.concatenate((new_lattice3, np.transpose([new_lattice3[:,1]])), axis = 1)       
    #looping through the new lattice 
    for ycor in range(len(new_lattice4)):
        for xcor in range(len(new_lattice4)):
            #this makes sure that only the points from the original lattice are counted
            if ycor >0 and ycor < len(new_lattice4)-1 and xcor >0 and xcor < len(new_lattice4[0])-1:
                #checks the energy in every direction and adds the contribution
                if new_lattice4[ycor][xcor] + new_lattice4[ycor-1][xcor] == 0:
                    energy += +0.5
                else:
                    energy += -0.5
                if new_lattice4[ycor][xcor] + new_lattice4[ycor+1][xcor] == 0:
                    energy += +0.5
                else:
                    energy += -0.5
                if new_lattice4[ycor][xcor] + new_lattice4[ycor][xcor-1] == 0:
                    energy += +0.5
                else:
                    energy += -0.5
                if new_lattice4[ycor][xcor] + new_lattice4[ycor][xcor+1] == 0:
                    energy += +0.5
                else:
                    energy += -0.5                
    return energy

def magnetisation(self):
    #sum() was assumed common knowledge
    magnetisation = np.sum(self.lattice)
    return magnetisation

<img src="Pics/Task4EnergyandMagnetisationMIKE123.png">

# Introduction to the Monte Carlo simulation

## Average Energies and Magnetisation

### Configuration of 100 spins and the estimated time it takes to evaluate the average magnetisation at one temperature

In a system of 100 spins there are $2^{100}$ configurations. In base 10 this approximates to about $1.27 * 10^{30}$ configurations. So dividing this number by the computational rate of $10^9$ configurations per seconds gives us a time of $1.28 * 10^{21}$ seconds or about $4 * 10^{13}$ years.

## Modifying IsingLattice.py

### Defining the montecarlostep function and the statistics function to obtain values for $<E>$ and $<M>$

In [ ]:
def montecarlostep(self, T):
    #saving the energy before the change
    energy0 = self.energy()
    magnet0 = self.magnetisation()
    #the following two lines will select the coordinates of the random spin for you
    random_i = np.random.choice(range(0, self.n_rows))
    random_j = np.random.choice(range(0, self.n_cols))
    #flipping the spins in the original lattice
    if self.lattice[random_i][random_j] == -1:
        self.lattice[random_i][random_j] += 2
    else:
        self.lattice[random_i][random_j] -= 2
    #saving the energy after the change
    energy1 = self.energy()
    magnet1 = self.magnetisation()
    energy_d = energy1 - energy0
    #the following line will choose a random number in the range [0,1) for you
    random_number = np.random.random()
    #checking whether the original lattice should have been kept (rather than checking if we should change)
    if energy_d > 0 and random_number > np.e**(-energy_d/T):
    #if yes then the spin is flipped back and the original energy is stored in energy1
        if self.lattice[random_i][random_j] == -1:
            self.lattice[random_i][random_j] += 2
        else:
            self.lattice[random_i][random_j] -= 2
        energy1 = energy0
        magnet1 = magnet0
    #always append the energy1 from the new system/corrected-back system
    self.E += energy1
    self.E2 += energy1**2
    self.M += magnet1
    self.M2 += magnet1**2
    self.n_cycles += 1
    return energy1, magnet1

def statistics(self):
    #divide by cycles, have an if condition so that it doesn't crash when n_cycles = 0
    if self.n_cycles > 0:
        avg_e = self.E/self.n_cycles
        avg_e2 = self.E2/self.n_cycles
        avg_m = self.M/self.n_cycles
        avg_m2 = self.M2/self.n_cycles
        return avg_e, avg_e2, avg_m, avg_m2, self.n_cycles
    else:
        return self.E, self.E2, self.M, self.M2, self.n_cycles 

### Discussing the the behaviour of a system when $T < T_C$

As given in the instructions, the system under investigation is set to be below the $T_C$, so that $T < T_C$ is fulfilled. In this case thermal energy is too low to influence any entropic forces, so that it is very unlikely that higher spin energy states are populated spontaneously. Thus what will happen is that the system tries to minimise it's energy by aligning all spins. This will lead to all spins being parallel and thus $<M> \neq 0$. This was also observed when running ILanim.py on the system:

<img src="Pics/360px-CMPWIKIPICTUREOFTHEENERGYSTEPS.png">


# Accelerating the code

### Duration of ILtimetrial.py for 2000  Monte Carlo steps
Duration for 2000 Monte Carlo Steps: One can safely assume the code executes within about 17.5 +/- 0.2 seconds.

### Improving the energy() and magnetisation() function


In [ ]:
def energy(self):       
    energy = 0.0
    s_ilattice = self.lattice
    #creates four lattices which are shifted with respect to s_ilattice in the four directions by one item (they wrap around edges)
    s_jlattice1 = np.roll(s_ilattice, 1, axis=0) #down
    s_jlattice2 = np.roll(s_ilattice, -1, axis=0) #up
    s_jlattice3 = np.roll(s_ilattice, 1, axis=1) #right
    s_jlattice4 = np.roll(s_ilattice, -1, axis=1) #left
    #they are added essentially creating a latticesum of neighbours for every item in s_ilattice
    s_jlatticesum = s_jlattice1 + s_jlattice2 + s_jlattice3 + s_jlattice4
    #the coordinates of an item in s_ilattice correspond to the coordinates of the sum of its neighbours in s_jlatticesum
    #multiplying them with each other gives s_i *s_j for all neighbours
    #taking the sum in order to get a total energy throughout the lattice
    energy += np.sum(np.multiply(s_ilattice, s_jlatticesum))*(-0.5)
    return energy

#as before
def magnetisation(self):
    magnetisation = np.sum(self.lattice)
    return magnetisation


### Duration of ILtimetrial.py for 2000  Monte Carlo steps after function improvement

Thus an average of about 0.43 +/- 0.1 seconds for the 2000 Monte Carlo Steps can be assumed. This is a major improvement of about 17 seconds. A major contributor to this improvement in run time is certainly the removal of for loops and the implementation of built in functions.

# The effect of temperature

## Correcting the Averaging Code

### Adjusting the code to account only for fluctuations around the equilibrium state

In [ ]:
# global variable has been implemented which can be altered at wish
ignored_cyc = 225000

def montecarlostep(self, T):
    energy0 = self.energy()
    magnet0 = self.magnetisation()
    random_i = np.random.choice(range(0, self.n_rows))
    random_j = np.random.choice(range(0, self.n_cols))
    if self.lattice[random_i][random_j] == -1:
        self.lattice[random_i][random_j] += 2
    else:
        self.lattice[random_i][random_j] -= 2
    energy1 = self.energy()
    magnet1 = self.magnetisation()
    energy_d = energy1 - energy0
    random_number = np.random.random()
    if energy_d > 0 and random_number > np.e**(-energy_d/T):
        if self.lattice[random_i][random_j] == -1:
            self.lattice[random_i][random_j] += 2
        else:
            self.lattice[random_i][random_j] -= 2
        energy1 = energy0
        magnet1 = magnet0
    #energies are only added if the number of cycles is larger than the number of ignored cycles
    if self.n_cycles > self.ignored_cyc:
        self.E += energy1
        self.E2 += energy1**2
        self.M += magnet1
        self.M2 += magnet1**2
    self.n_cycles += 1
    return energy1, magnet1

def statistics(self):
    if self.n_cycles > 0:
    #statistical values are calculated with the corrected number of cycles
        avg_cycles = self.n_cycles - self.ignored_cyc
        avg_e = self.E/avg_cycles
        avg_e2 = self.E2/avg_cycles
        avg_m = self.M/avg_cycles
        avg_m2 = self.M2/avg_cycles
        return avg_e, avg_e2, avg_m, avg_m2
    else:
        return self.E, self.E2, self.M, self.M2


The system was analysed for a grid of $16 \times 16$ and  $32 \times 32$ at temperatures of  $T = 2.0$ and  $T = 4.0$.
It was discovered that as the critical temperature lies somewhere between those two temperatures, as below  $T_C$ the system took time (up to 225000 steps) to reach equilibrium while when  $T > T_C$ the system had enough thermal energy to easily overcome the energy barrier and thus reach equilibrium really fast (less than 1000 steps) . The latter observation is due to the fact that a above the Curie Temperature random magnetisation is very unlikely to happen. 
It was further observed that at bigger lattice sizes the time it took for the system to reach equilibrium was larger.
It was thus concluded that in order to be prepared for the worst case scenario ($32 \times 32$ with $T$ just under $T_C$), a total of 225000 cycles will be ignored. This number can be obtained from observing the last graph.


<gallery widths=280px heights=200px>
<img src="Pics/280px-MonteCarlo0ignored2temp150000.png">|16x16 at T = 2.0, 150000 steps
<img src="Pics/280px-MonteCarlo0ignored4temp150000.png">|16x16 at T = 4.0, 150000 steps
<img src="Pics/280px-MonteCarlo32320ignored2temp500000.png">|32x32 at T = 2.0, 500000 steps
<img src="Pics/280px-MonteCarlo32320ignored4temp500000.png">|32x32 at T = 4.0, 500000 steps
</gallery>

## Running over a range of temperatures

### Average Energy and Magnetisation for each Temperature 

It was decided that the simulations will run in the temperature range of $ T = 0.5 $ and $ T=5.0 $ in steps of $ dT = 0.05$ for a total number of 350000 steps with 225000 steps being disregarded in the energy and magnetisation averages. The results are presented in the next section.

# The effect of system size
## Scaling the System Size
### Energy per spin versus temperature diagrams for different lattice sizes 

In order to plot the functions from the obtained data files the following function was defined to be called for every lattice size and the output is shown below.


In [ ]:
import numpy as np
from matplotlib import pylab as plt


def plotfunct(filename, lattice_side_size):
    data = np.loadtxt(filename)
    temps = data[:,0]
    energies=data[:,1]
    energies2=data[:,2]
    magnet=data[:,3]
    magnet2=data[:,4]
    spins = lattice_side_size**2
#errorbars as square root of variance, energies are given per spin
    energies_error = np.sqrt(energies2-energies**2)/spins
    magnet_error = np.sqrt(magnet2-magnet**2)/spins

    plt.ylabel("Energy per spin")
    plt.xlabel("Temperature")
    plt.title('Energy per spin vs Temperature (' + str(lattice_side_size)+'x'+str(lattice_side_size)+')')
    plt.errorbar(temps, energies/spins,yerr = energies_error, ecolor = "green")
    plt.show()
    plt.ylabel("Magnetisation per spin")
    plt.xlabel("Temperature")
    plt.title('Magnetisation per spin vs Temperature (' + str(lattice_side_size)+'x'+str(lattice_side_size)+')')
    plt.errorbar(temps, magnet/spins, yerr = magnet_error, ecolor = "green")
    pl.show()

As lattice size increases, fluctuations and error bars decrease massively. It is further observed that the point at which equilibrium is reached in the  magnetisation graph shifts from left to right (to increasing temperature) as lattice size is increasing. The first observation can be explained through the way data is processed. In small systems like 2x2, flipping of one spin has a huge impact on the absolute change in energy (1 in 4 spins), whereas the same flip in a 32x32 system has a minor contribution (1 in 1024). Latter will thus have a much smaller impact on the statistical means that are used to obtain the graphs. The second observation can also be explained through the quantity of spins in each lattice. As the Monte Carlo algorithm assigns multiple random numbers when it is called, it is much more likely to totally reverse a small system than a large one. I.e. completely reversing a system is much more likely to happen in a 2x2 lattice (4 different flips needed) versus a 32x32 lattice (1024 different flips needed!!!).

The quality of the data (reduced noise) does not seem to linearly depend on lattice size (judged by eye). However it does definitely increase with increasing lattice size. While 32x32 seems adequate to perform calculations with, 64x64 would very likely improve the data even more.

<img src="Pics/320px-Xxx2x2E.png">
<img src="Pics/320px-Xxx4x4E.png">
<img src="Pics/320px-Xxx8x8E.png">
<img src="Pics/320px-Xxx16x16E.png">
<img src="Pics/320px-Xxx32x32E.png">
<img src="Pics/320px-Xxx2x2M.png">
<img src="Pics/320px-Xxx4x4M.png">
<img src="Pics/320px-Xxx8x8M.png">
<img src="Pics/320px-Xxx16x16M.png">
<img src="Pics/320px-Xxx32x32M.png">


# Determining the heat capacity
## Calculating the heat Capacity
### Deriving an expression for heat capacity from the partial derivative expression

1. The heat capacity is given as $C = \frac{\partial \left\langle E\right\rangle}{\partial T}$ , with 
$\left\langle E\right\rangle = \frac{\sum_\alpha E\left(\alpha\right) \exp \left\{-\frac{E\left(\alpha\right)}{k_\mathrm BT}\right\}}{\sum_\alpha  \exp \left\{-\frac{E\left(\alpha\right)}{k_\mathrm BT}\right\}}$ 

2. Defining $Z=\sum_\alpha  \exp \left\{-\frac{E\left(\alpha\right)}{k_\mathrm BT}\right\}$ and $y= \frac{1}{k_\mathrm BT}$ for simplification purposes and noting that $\frac{\partial y}{\partial T} = -\frac{1}{k_BT^2}$

3. The expression which should be derived is $C = \frac{\mathrm{Var}[E]}{k_B T^2}$, where $\mathrm{Var}[E] = \left\langle E^2\right\rangle - \left\langle E\right\rangle ^2 $. 

4. So from that an expression for $\left\langle E\right\rangle $ and $\left\langle E^2\right\rangle $ in terms of $Z$ and $y$ shall be found.

5. Differentiating $ Z $ with respect to $y$ gives:
$\frac{\partial Z}{\partial y}= - \sum_\alpha E\left(\alpha\right) \exp \left\{-yE\left(\alpha\right)\right\} $.

6. Using this information $\left\langle E\right\rangle $ can be rewritten as  $\left\langle E\right\rangle = -\frac{1}{Z}\frac{\partial Z}{\partial y}$

7. Differentiating $ Z $ with respect to $y$ twice gives:
$\frac{\partial^2 Z}{\partial y^2} = \sum_\alpha E\left(\alpha\right)^2 \exp \left\{-\frac{E\left(\alpha\right)}{k_\mathrm BT}\right\} $

8. Thus $\left\langle E^2\right\rangle $ can be rewritten as $\left\langle E^2\right\rangle = \frac{1}{Z} \frac{\partial^2 Z}{\partial y^2}  $

9. Evaluating the original expression using the product rule gives:
$C  = \frac{\partial}{\partial T} \left(- \frac {1}{Z} \frac{\partial Z}{\partial y}\right) =  \frac{\partial \left(-\frac{1}{Z}\right)}{\partial T} \frac{\partial Z}{\partial y} - \frac{1}{Z} \frac{\partial}{\partial T} \left( \frac{\partial Z}{\partial y}\right) $

10. Applying the chain rule twice in the first term and once in the second term gives:
$ C =  \frac{\partial \left(-\frac{1}{Z}\right)}{\partial Z} \frac{\partial Z}{\partial y} \frac{\partial y}{\partial T}\frac{\partial Z}{\partial y} - \frac{1}{Z} \frac{\partial \left( \frac{\partial Z}{\partial y}\right)}{\partial y}\frac{\partial y}{\partial T} =\frac{\partial y}{\partial T} \left( \left(\frac{1}{Z}\frac{\partial Z}{\partial y}\right)^2-\frac{1}{Z}\frac{\partial ^2Z}{\partial y^2}\right)$

11. Substituting the expressions for the partial derivatives from 2., 6. and 8., hence gives:
$C = \frac{\left\langle E^2\right\rangle - \left\langle E\right\rangle ^2} {k_B T^2}$


### Plotting Heat Capacity vs Temperature for different lattice sizes


In [ ]:
def plotheatcap(filename, lattice_side_size):
    data = np.loadtxt(filename)
    temps = data[:,0]
    spins = lattice_side_size**2
    energies=data[:,1]
    energies2=data[:,2]
    varian = (energies2-energies**2)/spins

    plt.ylabel("Heat Capacity")
    plt.xlabel("Temperature")
    plt.plot(temps, varian/temps**2, label = str(lattice_side_size) + "x"+ str(lattice_side_size))
    

def plotmultiple(filenamearray, lattice_side_array):
    for i in range(len(filenamearray)):
        plotheatcap(filenamearray[i], lattice_side_array[i])
    plt.title("Heat capactiy per spin vs temperature for various lattice sizes")
    plt.legend(loc="best",title="Lattice size",fontsize="8")
    plt.show()
    
plotmultiple(['2x2.dat','4x4.dat','8x8.dat','16x16.dat','32x32.dat'],[2,4,8,16,32])


This code leads to the graph below. The heat capacity per spin seem to slightly increase with the size of the lattice. The peak does also seem to become more narrow. This directly relates to the previous discussion about lattice size affecting statistical measures, as heat capacity is directly related to the variance of the obtained energies. I.e. larger spins system lead to smaller error / variance and thus a more narrow peak of the heat capacity. Ideally at very large lattice size this peak will become as narrow as possible.
 

<img src="Pics/500px-HeatCapxxx123.png">

# Locating the Curie temperature
## C++
### Comparing the quality of data obtained using python versus C++

In [ ]:
def plottwocap(filenamearray, lattice_side_array):
    plotheatcap(filenamearray[0],lattice_side_array[0])

    data2 = np.loadtxt(filenamearray[1])
    temps2 = data2[:,0]
    spins2 = lattice_side_array[1]**2
    energies2=data2[:,1]
    energies22=data2[:,2]
    varian2 = (energies22-energies2**2)
    capa = data2[:,5]
    
    plt.ylabel("Heat Capacity")
    plt.xlabel("Temperature")
    plt.plot(temps2, capa, label = "C++")
    plt.title("Heat capactiy per spin vs temperature (32x32)")
    plt.legend(loc="best",title="Lattice size",fontsize="8")
    plt.show()

def plottwo2(filenamearray, lattice_side_array):
    data1 = np.loadtxt(filenamearray[0])
    temps1 = data1[:,0]
    spins1 = lattice_side_array[0]**2
    energies = data1[:,1]
    magnet = data1[:,3]

    data2 = np.loadtxt(filenamearray[1])
    temps2 = data2[:,0]
    energies2=data2[:,1]
    magnet2 = data2[:,3]
    
    plt.ylabel("Energy per spin")
    plt.xlabel("Temperature")
    plt.title('Energy per spin vs Temperature (' + str(lattice_side_array[0])+'x'+str(lattice_side_array[0])+')')
    plt.plot(temps1, energies/spins1,label="python")
    plt.plot(temps2, energies2,label="C++")
    plt.legend(loc="best",title="Language",fontsize="8")
    plt.show()
    
    plt.ylabel("Magnetisation per spin")
    plt.xlabel("Temperature")
    plt.title('Magnetisation per spin vs Temperature (' + str(lattice_side_array[0])+'x'+str(lattice_side_array[0])+')')
    plt.plot(temps1, magnet/spins1,label="python")
    plt.plot(temps2, magnet2,label="C++")
    plt.legend(loc="best",title="Language",fontsize="8")
    plt.show()
    
plottwo2(['32x32.dat','C++_data/32x32C.dat'],[32,32])

plottwocap(['32x32.dat','C++_data/32x32C.dat'],[32,32])

The data from the C++ files was collected in steps of $ dT = 0.01T $ rather than $ dT = 0.05T $ as was done in the previous section. This lead to a higher data rate and higher resolution in all graphs. It was especially noticeable in the region around the Curie Temperature for the magnetisation per spin graph as fluctuations were made visible which were originally assumed completely absent.

<img src="Pics/300px-ENEEE.png">
<img src="Pics/300px-MAGMM.png">
<img src="Pics/300px-HEATTT.png">

## Polynomial fitting
### Polyfitting Heat Capacity to a polynomial


In [ ]:
def plotpolyfit(filename):
    dataA = np.loadtxt(filename)
    temps = dataA[:,0]
    capa = dataA[:,5]
    
    fit = np.polyfit(temps,capa,5)
    fit2 = np.polyfit(temps,capa,100)
    T_min=np.min(temps)
    T_max=np.max(temps)
    T_range=np.linspace(T_min,T_max,1000)
    fitted_C_values = np.polyval(fit,T_range)
    fitted_C_values2 = np.polyval(fit2,T_range)

    plt.plot(temps,capa,label='Collected Data')
    plt.plot(T_range,fitted_C_values,label='Fitted Data (Degree 5)')
    plt.plot(T_range,fitted_C_values2,label='Fitted Data (Degree 100)')
    plt.legend(loc ="best", fontsize= 8)
    plt.title('Heat Capacity per Spin Curve Fitting (32x32)')
    plt.xlabel('Temperature')
    plt.ylabel("Heat Capacity")
    plt.show()

plotpolyfit('C++_data/32x32C.dat')

Some initial trials to polyfit a polynomial to the whole data range only showed little success. Even when increasing the polynomial degree to 100, it was not possible to approximate the peak accurately.
<img src="Pics/500px-Adadasdasdasdc.png">

### Polyfit the heat capacity to a polynomial in a specific temperature range


In [ ]:
def plotpolyfitinrange(filename, dimension, minT, maxT, degree):
    dataA = np.loadtxt(filename)
    temps = dataA[:,0]
    capa = dataA[:,5]

    selection = np.logical_and(temps > minT, temps < maxT) #choose only those rows where both conditions are true
    peak_T_values = temps[selection]
    peak_C_values = capa[selection]
    
    fit = np.polyfit(peak_T_values,peak_C_values,degree)
    T_min=np.min(temps)
    T_max=np.max(temps)
    T_range=np.linspace(T_min,T_max,1000)
    fitted_C_values = np.polyval(fit,T_range)
    fitted_C_values2 = np.polyval(fit,peak_T_values)
    Cmax = np.max(fitted_C_values2)
    Tmax = peak_T_values[fitted_C_values2 == Cmax]
    print(Cmax, Tmax)
    plt.plot(temps,capa,label='Collected Data')
    plt.plot(T_range,fitted_C_values,label='Fitted Data between ['+ str(minT) +','+str(maxT)+'] (Degree '+str(degree)+')')
    plt.legend(loc ="best", fontsize= 8)
    plt.title('Heat Capacity per Spin Curve Fitting (' + str(dimension)+'x'+str(dimension)+')')
    plt.xlabel('Temperature')
    plt.ylabel("Heat Capacity")
    plt.ylim([-0.5,2])
    plt.show()

plotpolyfitinrange('C++_data/2x2C.dat',2,1.5,3.4,4)
plotpolyfitinrange('C++_data/4x4C.dat',4,1.5,3.4,4)
plotpolyfitinrange('C++_data/8x8C.dat',8,1.7,3,6)
plotpolyfitinrange('C++_data/16x16C.dat',16,2,2.7,6)
plotpolyfitinrange('C++_data/32x32C.dat',32,2.2,2.4,5)

A big improvement was made when small regions were selected to run through the polyfit function. The last parameter in the function "plotpolyfztinrange" above indicates the polynomial degree used for further calculations. By reducing the domain of the function, even polynomial of degree 4 were enough to accurately fit the heat capacity data (for 2z2 and 4x4). 

<img src="Pics/500px-POLYFIT2njsdnfksndk.png">

### Plotting the scaling relation between the Curie Temperature and the lattice side length

The data for the table below was obtained when executing the previous function for every lattice. It showed that the Curie Temperature decreased with increasing lattice size as was expected in the provided function of $T_{C, L} = \frac{A}{L} + T_{C,\infty}$. 

In order to obtain the Currier temperature at infinite lattice the temperatures (table above) at which the heat capacities were maximised were plotted against the inverse of the respective lattice side length, $\frac{1}{L}$.

In [ ]:
sidelength = np.array([2,4,8,16,32])
curieT = np.array([2.52,2.48,2.36,2.32,2.30])

fit = np.polyfit(1/sidelength, curieT, 1, cov=True)
params = fit[0]
params_cov = fit[1] 

x_range = np.linspace(0, 0.5, 1000) 
fitted_curie = params[0]*x_range + params[1]
plt.xlabel("1/L")
plt.ylabel("Curie temperature")
plt.plot(1/sidelength, curieT, linestyle='',marker='x',label="Collected Data")
plt.plot(x_range, fitted_curie, label="Linear fit")
plt.title("Curie temperature versus 1/L for LxL lattices")
plt.legend(loc="best",fontsize=10)
plt.show()

<img src="Pics/500px-Sclaingrelation123123.png">

A value of $ 2.301 \pm 0.044$ for the Currier Temperature at infinite lattice was obtained (y-intercept). This result lies within the boundaries of the literature [1] value of 2.269. However, when analysing the graph below it seems quite obvious that the point at 2x2 is an outlier. The reasons for that were discussed thoroughly in previous sections above as the uncertainty of the 2x2 lattice's data continuously showed up throughout this report. 
Additional minor factors that contribute to this deviation from the literature value will come from the method used, i.e. the Monte Carlo algorithm rejects complete randomness and is thus an approximation. However, it is actually rather a personal surprise that such a good close result was obtained using this algorithm, that cut down run time from about $10^{13}$ years to just around 30 minutes. So essentially having a fairly big lattice size of about $128 \times 128$ should be ideal to increase accuracy while sacrificing little additional computational time. 

## References
<references>
[1] <ref name="Gangsta">L. Onsager, Crystal Statistics. I. A Two-Dimensional Model with an Order-Disorder Transition, <i>Physical Review</i>, 1944, <b>65</b>, 117-149.</ref>
</references>
